In [ ]:
for j in range(1,22) :
            folium.CircleMarker(location=[a.iloc[3,j], a.iloc[3,j+1]], radius=0.05,
                    color='green', fill_color='#0080bb').add_to(map)          
        
        
for i in range(round(len(y)*0.7)) :        
              folium.CircleMarker(location=[y.iloc[i,1], y.iloc[i,2]], radius=0.05,
                        color='red', fill_color='#0080bb').add_to(map) 
                
        
map.save('C:/Users/renz/Desktop/green.html')  

# prediction

In [ ]:
port_id = 'MXZLO'
port_ais = port_5km(data=data, lat=df2.loc[df2.port_id.tolist().index(port_id), 'LAT'], lon=df2.loc[df2.port_id.tolist().index(port_id), 'LON'])
df3 = ten_number(port_ais)
kmeans_clustering(port_ais_data=port_ais, ten_number_data=df3)

cluster_data = kmeans_clustering_receive(port_ais_data=port_ais, ten_number_data=df3)
cluster_data = pd.concat([cluster_data.fillna(2).groupby('MMSI3').mean().reset_index().MMSI3, cluster_data.fillna(2).groupby('MMSI3').mean().reset_index().iloc[:,1:].apply(lambda x: ''.join(x.astype(int).astype(str)), axis=1)], axis=1)
cluster_data.rename(columns={0:'Cluster'}, inplace=True)
cluster_data.Cluster.unique()

cluster_data[cluster_data.Cluster=='02222022222']

test_receive(cluster_data)

x,y = train_test_set_receive(cluster_data, '477157700_6_2', '02222022222')
x1 = y_70(x)

# true_y = y_70(y).loc[y_70(y).index.tolist().index(max(y_70(y).index.tolist()))-1,'y_value']
y1 = y_70(y)[['MMSI3', 'LAT', 'LON', 'y_value', 'distance']]

train_x = ten_number_for_model(x1).drop([0,23], axis=1); train_y = ten_number_for_model(x1)[23]

y1['y_value'] = y1['y_value'].apply(lambda x: x * randrange(9000000, 11000000)/10000000)

test_x = ten_number_for_model(y1).drop([0, 23], axis=1); test_y = ten_number_for_model(y1)[23]

In [ ]:
###############################################################################################################
# 모델링 및 예측 (다중 선형 회귀 분석)
reg = LinearRegression()
reg.fit(train_x, train_y)
y_pred = reg.predict(test_x)
abs(y_pred - test_y)
###############################################################################################################

###############################################################################################################
# create dataset for lightgbm
lgb_train = lgb.Dataset(train_x, train_y)
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 10,
    'learning_rate': 0.05,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.5,
    'bagging_freq': 5,
    'min_child_samples' : 1,
    'verbose': 0
    
}
gbm = lgb.train(params, lgb_train, num_boost_round=119)
# predict
y_pred = gbm.predict(test_x, num_iteration=gbm.best_iteration)
abs(y_pred-test_y)
###############################################################################################################

###############################################################################################################
# 모델링 및 예측 (xgboost)
y_pred = xgb.XGBRegressor(booster='gblinear', eta = 10, min_child_weight =22,
                          max_depth = 8, gamma = 100 , alpha = 0.1, n_estimators=10000).fit(train_x, train_y).predict(test_x)

abs(test_y-y_pred)
###############################################################################################################

y_pred = RandomForestRegressor(n_estimators = 100).fit(train_x, train_y).predict(test_x)

predict_result = pd.DataFrame()
df_reg = pd.DataFrame()
df_xg = pd.DataFrame()
df_rf = pd.DataFrame()


for i in df2['port_id']:
    port_id = i
    port_ais = port_5km(data=data, lat=df2.loc[df2.port_id.tolist().index(port_id), 'LAT'], lon=df2.loc[df2.port_id.tolist().index(port_id), 'LON'])
    df3 = ten_number(port_ais)
#     kmeans_clustering(port_ais_data=port_ais, ten_number_data=df3)
    cluster_data = kmeans_clustering_receive(port_ais_data=port_ais, ten_number_data=df3)
    cluster_data = pd.concat([cluster_data.fillna(2).groupby('MMSI3').mean().reset_index().MMSI3, cluster_data.fillna(2).groupby('MMSI3').mean().reset_index().iloc[:,1:].apply(lambda x: ''.join(x.astype(int).astype(str)), axis=1)], axis=1)
    cluster_data.rename(columns={0:'Cluster'}, inplace=True)

#         cluster_data.Cluster.unique()

    df1 = test_receive(cluster_data)

    for j1, j2 in zip(df1['MMSI3'], df1['Cluster']):
        x,y = train_test_set_receive(cluster_data, j1, j2)
        x1 = y_70(x)
        y1 = y_70(y)[['MMSI3', 'LAT', 'LON', 'y_value', 'distance']]
        train_x = ten_number_for_model(x1).drop([0,23], axis=1); train_y = ten_number_for_model(x1)[23]
        y1['y_value'] = y1['y_value'].apply(lambda x: x * randrange(9000000, 11000000)/10000000)
        test_x = ten_number_for_model(y1).drop([0, 23], axis=1); test_y = ten_number_for_model(y1)[23]
        
        train_x.columns = ['LAT11','LON12','arrive_time13','distance14',  'LAT21','LON22','arrive_time23','distance24', 'LAT31','LON32','arrive_time33','distance34', 'LAT41','LON42','arrive_time43','distance44', 'LAT51','LON52','arrive_time53','distance54', 'LAT61','LON62','distance64', 'LAT71','LON72']
        test_x.columns = ['LAT11','LON12','arrive_time13','distance14',  'LAT21','LON22','arrive_time23','distance24', 'LAT31','LON32','arrive_time33','distance34', 'LAT41','LON42','arrive_time43','distance44', 'LAT51','LON52','arrive_time53','distance54', 'LAT61','LON62','distance64', 'LAT71','LON72']
        
        train_x = train_x.loc[:,['LAT11','LON12','LAT21','LON22', 'LAT31','LON32', 'LAT41','LON42', 'LAT51','LON52', 'LAT61','LON62', 'LAT71','LON72']]
        test_x = test_x.loc[:,['LAT11','LON12','LAT21','LON22', 'LAT31','LON32', 'LAT41','LON42', 'LAT51','LON52', 'LAT61','LON62', 'LAT71','LON72']]
    
        
        # Regression
        reg = LinearRegression()
        reg.fit(train_x, train_y)
        y_pred = reg.predict(test_x)
        error1 =  test_y - y_pred
        
        df_reg = pd.concat([df_reg, pd.DataFrame(reg.coef_)], axis=1)

        
        
        try:
            # create dataset for lightgbm
            lgb_train = lgb.Dataset(train_x, train_y)
            # specify your configurations as a dict
            params = {
                'boosting_type': 'gbdt',
                'objective': 'regression',
                'metric': {'l2', 'l1'},
                'num_leaves': 10,
                'learning_rate': 0.05,
                'feature_fraction': 0.3,
                'bagging_fraction': 0.5,
                'bagging_freq': 5,
                'min_child_samples' : 1,
                'min_data_in_bin' : 1,
                'verbose': 0

            }
            gbm = lgb.train(params, lgb_train, num_boost_round=119)
            # predict
            y_pred = gbm.predict(test_x, num_iteration=gbm.best_iteration)
            error2 = test_y - y_pred
        except:
            error2 = 'small data'

        # 모델링 및 예측 (xgboost)
        y_pred = xgb.XGBRegressor(booster='gblinear', eta = 10, min_child_weight =22,
                                  max_depth = 8, gamma = 100 , alpha = 0.1, n_estimators=10000).fit(train_x, train_y).predict(test_x)
        error3 = test_y-y_pred
        
        df_xg = pd.concat([df_xg, pd.DataFrame(xgb.XGBRegressor(booster='gblinear', eta = 10, min_child_weight =22,
                                  max_depth = 8, gamma = 100 , alpha = 0.1, n_estimators=10000).fit(train_x, train_y).coef_)], axis=1)
        
        # random forest
        y_pred = RandomForestRegressor(n_estimators = 100).fit(train_x, train_y).predict(test_x)

        error4 = test_y-y_pred
        df_rf = pd.concat([df_rf, pd.DataFrame(RandomForestRegressor(n_estimators = 100).fit(train_x, train_y).feature_importances_)], axis=1)
        
        
        predict_result = pd.concat([predict_result, pd.DataFrame([i, j1, j2, error1, error2, error3, error4])], axis=1)